# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core import Workspace, Experiment, Environment, Model
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

from azureml.automl.runtime.onnx_convert import OnnxConverter
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

from azureml.core import Webservice
# Check core SDK version number


import requests
import json
print("SDK version:", azureml.core.VERSION)

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'EXP_HEART_FAILURE_AML'

experiment=Experiment(ws, experiment_name)

In [3]:
# Building CPU cluster
amlcompute_cluster_name = "auto-ml"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           min_nodes=4,
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

Creating.........
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded........................
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [4]:
found = False
key = "heart-failure" #"Heart failure"
description_text = "Heart failure dataset for udacity capstone"

if key in ws.datasets.keys():
        found = True
        print("Found dataset: ",key)
        dataset = ws.datasets[key]

if not found:
        # Create AML Dataset and register it into Workspace
        print("Registering dataset: ",key)
        example_data = "./data/heart_failure_clinical_records_dataset.csv"
        dataset = Dataset.Tabular.from_delimited_files(example_data)
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

Found dataset:  heart-failure


,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000,299.000000,299.00000
mean,60.833893,0.431438,581.839465,0.418060,38.083612,0.351171,263358.029264,1.39388,136.625418,0.648829,0.32107,130.260870,0.32107
std,11.894809,0.496107,970.287881,0.494067,11.834841,0.478136,97804.236869,1.03451,4.412477,0.478136,0.46767,77.614208,0.46767
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.50000,113.000000,0.000000,0.00000,4.000000,0.00000
25%,51.000000,0.000000,116.500000,0.000000,30.000000,0.000000,212500.000000,0.90000,134.000000,0.000000,0.00000,73.000000,0.00000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.10000,137.000000,1.000000,0.00000,115.000000,0.00000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303500.000000,1.40000,140.000000,1.000000,1.00000,203.000000,1.00000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.40000,148.000000,1.000000,1.00000,285.000000,1.00000


In [5]:
dataset.take(5).to_pandas_dataframe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [6]:
project_folder='./realtime_automl'
automl_settings = {
    "experiment_timeout_minutes": 15,
    "max_concurrent_iterations": 4,
    "primary_metric" : 'AUC_weighted'
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             model_explainability = True,
                             enable_onnx_compatible_models=True,
                             **automl_settings
                            )

In [7]:
automl_run1 = experiment.submit(automl_config, show_output=True)

Submitting remote run.
No run_configuration provided, running on auto-ml with default configuration
Running on remote compute: auto-ml


Experiment,Id,Type,Status,Details Page,Docs Page
EXP_HEART_FAILURE_AML,AutoML_03202041-e1a9-4c4a-b446-9912b5836698,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+---------------------------------+
|Number of folds                  |
+=================================+
|10                               |
+---------------------------------+

****************************************************************************************************

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

*************************************************

In [10]:
rns = experiment.get_runs()
next(rns)

Experiment,Id,Type,Status,Details Page,Docs Page
EXP_HEART_FAILURE_AML,AutoML_03202041-e1a9-4c4a-b446-9912b5836698,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [8]:
from azureml.widgets import RunDetails
widget1 = RunDetails(automl_run1)
widget1.show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [11]:
best_run = automl_run1.get_best_child()
best_run.get_metrics()

{'recall_score_macro': 0.8322528963218323,
 'recall_score_weighted': 0.8593103448275862,
 'f1_score_weighted': 0.8565721467762923,
 'norm_macro_recall': 0.6645057926436644,
 'log_loss': 0.3649091409216298,
 'precision_score_micro': 0.8593103448275862,
 'f1_score_micro': 0.8593103448275862,
 'AUC_weighted': 0.920757517008375,
 'matthews_correlation': 0.6854341113441387,
 'recall_score_micro': 0.8593103448275862,
 'average_precision_score_macro': 0.9150195036408331,
 'accuracy': 0.8593103448275862,
 'balanced_accuracy': 0.8322528963218323,
 'AUC_micro': 0.9263472057074912,
 'weighted_accuracy': 0.8761533052789211,
 'precision_score_weighted': 0.8736864816537231,
 'average_precision_score_micro': 0.9300646414652924,
 'AUC_macro': 0.920757517008375,
 'precision_score_macro': 0.8559815046065046,
 'average_precision_score_weighted': 0.9363472810342028,
 'f1_score_macro': 0.8321523872315162,
 'confusion_matrix': 'aml://artifactId/ExperimentRun/dcid.AutoML_03202041-e1a9-4c4a-b446-9912b5836698_

In [12]:
print('Besr run id : ', best_run.id)

Besr run id :  AutoML_03202041-e1a9-4c4a-b446-9912b5836698_51


In [15]:
#TODO: Save the best model
best_automl_model, onnx_model = automl_run1.get_output(return_onnx_model=True)
OnnxConverter.save_onnx_model(onnx_model, './automl_model.onnx')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [25]:
model = best_run.register_model(model_path='outputs/model.onnx', model_name='mdl-heart-failure-automl-onnx')

In [ ]:
best_automl_model.

In [22]:
myenv = Environment.get(workspace=ws, name="AzureML-Scikit-learn0.24-Cuda11-OpenMpi4.1.0-py36")
script_path = './result/scoring_file_v_2_0_0.py'
inference_config = InferenceConfig(entry_script=script_path, 
environment=myenv)

In [23]:
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

In [26]:
service = Model.deploy(
    workspace=ws,
    name="mdldply-heart-failure-aci",
    models=[model],
    inference_config=inference_config,
    deployment_config=aci_config,
    overwrite=True
)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-05-29 13:37:44+00:00 Creating Container Registry if not exists.
2021-05-29 13:37:45+00:00 Registering the environment.
2021-05-29 13:37:46+00:00 Use the existing image.
2021-05-29 13:37:46+00:00 Generating deployment configuration.
2021-05-29 13:37:47+00:00 Submitting deployment to compute.
2021-05-29 13:37:50+00:00 Checking the status of deployment mdldply-heart-failure-aci..
2021-05-29 13:51:24+00:00 Checking the status of inference endpoint mdldply-heart-failure-aci.
Failed


Service deployment polling reached non-successful terminal state, current service state: Unhealthy
Operation ID: c2bd5022-7fc7-43dc-abe1-c4001de3e739
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Error in entry script, ModuleNotFoundError: No module named 'azureml.automl', please run print(service.get_logs()) to get details.",
  "details": [
    {
      "code": "CrashLoopBackOff",
      "message": "Error in entry script, ModuleNotFoundError: No module named 'azureml.automl', please run print(service.get_logs()) to get details."
    }
  ]
}



WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Unhealthy
Operation ID: c2bd5022-7fc7-43dc-abe1-c4001de3e739
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Error in entry script, ModuleNotFoundError: No module named 'azureml.automl', please run print(service.get_logs()) to get details.",
  "details": [
    {
      "code": "CrashLoopBackOff",
      "message": "Error in entry script, ModuleNotFoundError: No module named 'azureml.automl', please run print(service.get_logs()) to get details."
    }
  ]
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Service deployment polling reached non-successful terminal state, current service state: Unhealthy\nOperation ID: c2bd5022-7fc7-43dc-abe1-c4001de3e739\nMore information can be found using '.get_logs()'\nError:

In [28]:
print(service.get_logs())

None


In [ ]:
name="mdldply-heart-failure-aci"
service = Webservice(name=name, workspace=ws)
service.update(enable_app_insights=True)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
data_json = {
	"data":
	[
		{"age": 75, "anaemia":0, "creatinine_phosphokinase": 582, "diabetes": 0, "ejection_fraction": 20, "high_blood_pressure": 1, "platelets": 265000, "serum_creatinine": 1.9, "serum_sodium": 130, "sex": 1, "smoking": 0, "time":4}
	]

}

In [ ]:
input_data = json.dumps(data_json)
with open("data.json", "w" ) as _f:
    _f.write(input_data)
headers =  {'Content-Type': 'application/json'}
headers['Authorization'] = f'Bearer {key}'

resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.json())

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
logs = service.get_logs()

for line in logs.split('\n'):
    print(line)

In [ ]:
compute_target.delete()